In [3]:
from transformers import (
    AutoTokenizer,
    BartForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    EarlyStoppingCallback
)
from tokenizers import Tokenizer
from typing import Dict, List, Optional
from torch.utils.data import Dataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import display
from typing import Dict

In [4]:
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration
model_name = "jjae/kobart-hashtag"
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [5]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [6]:
model = model.to(device)

In [7]:
def make_tag(text):
  # 입력 문장을 토큰화하여 인코딩
  input_ids = tokenizer.encode(text, return_tensors="pt").to(device)

  # 모델에 입력 전달하여 디코딩
  output = model.generate(input_ids = input_ids, bos_token_id = model.config.bos_token_id,
                          eos_token_id = model.config.eos_token_id, length_penalty = 2.0, max_length = 50, num_beams = 2)

  # 디코딩된 출력을 토크나이저를 사용하여 텍스트로 변환
  decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
  return decoded_output

In [9]:
text = "성공과 실패의 경험을 통해 나는 자신감과 겸손, 더 나은 방향을 찾기 위한 노력하는 자세를 얻을 수 있었다. 성공과 실패는 나에게 자기 인식과 성숙에 대한 깊은 생각을 하게 하며, 나를 더 나은 사람으로 성장시키는 보람 있는 과정이었다. "
make_tag(text)

'#성공과실패 #자신감과겸손 #도전과노력 #성장과발전 #자기인식 #성장과발전'

In [63]:
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration
model_name = "jjae/summary"
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)
model1 = BartForConditionalGeneration.from_pretrained(model_name).to(device)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [64]:
model1 = model1.to(device)

In [65]:
def diary_summary(text):
  # Encode input text
  input_ids = tokenizer.encode(text, return_tensors = 'pt').to(device)
  # Generate summary text ids
  summary_text_ids = model1.generate(input_ids = input_ids,
                                    bos_token_id = model.config.bos_token_id,
                                    eos_token_id = model.config.eos_token_id,
                                    length_penalty = 2.0,
                                    max_length = 150,
                                    num_beams = 2)
  return tokenizer.decode(summary_text_ids[0], skip_special_tokens = True)

In [66]:
diary = """새해 첫날, 설레는 기분으로 눈을 떴다. 햇살이 창밖으로 들어오는 모습을 보며 새로운 시작에 대한 기대와 희망이 가득했다. 목표와 꿈을 돌아보고 계획을 세웠다. 가족과 함께 산책을 즐기며 소중한 시간을 보냈고, 친구들과의 파티에서 즐거운 추억을 만들었다. 고마움과 감사함을 느끼며 새로운 365일에 대한 희망과 열정으로 가득한 하루였다."""

In [67]:
diary

'새해 첫날, 설레는 기분으로 눈을 떴다. 햇살이 창밖으로 들어오는 모습을 보며 새로운 시작에 대한 기대와 희망이 가득했다. 목표와 꿈을 돌아보고 계획을 세웠다. 가족과 함께 산책을 즐기며 소중한 시간을 보냈고, 친구들과의 파티에서 즐거운 추억을 만들었다. 고마움과 감사함을 느끼며 새로운 365일에 대한 희망과 열정으로 가득한 하루였다.'

In [68]:
diary_summary(diary)

'새로운 365일에 대한 희망과 열정으로 가득한 하루였다. 목표와 계획을 세우며 시간을 보냈고, 가족과 함께 산책을 즐기며 즐거운 시간을 보냈다.'

In [69]:
diary2 = '''1년이라는 시간 동안 개발자로서 회사에 다니며 많은 경험과 성장을 한 해였다. 첫날부터 회사의 분위기와 동료들의 친절함에 감동받았고, 새로운 기술과 프로젝트에 도전하는 과정에서 많은 것을 배웠다. 처음에는 많은 어려움과 새로운 지식에 대한 부담을 느꼈지만, 동료들의 도움과 지도를 받으며 조금씩 성장해나갔다. 프로젝트마다 다양한 도구와 언어를 사용하면서 자신의 역량을 끌어올리는 기회가 되었다. 또한, 팀원들과의 협업을 통해 문제를 해결하고 목표를 달성하는 과정에서 많은 것을 배웠다. 한 해 동안 회사에서의 업무와 개발 프로세스에 대해 많은 경험을 쌓았다. 프로젝트 기획부터 개발, 테스트, 배포까지의 과정을 경험하면서 개발자로서의 역할과 책임감을 느낄 수 있었다. 또한, 버그를 해결하거나 기능을 개선하는 과정에서 문제 해결 능력을 키워나갈 수 있었다. 회사 생활을 하면서 가장 중요한 것은 항상 새로운 것을 배우고 개발자로서 성장하는 것이라는 것을 느꼈다. 기술은 빠르게 변하기 때문에 항상 새로운 도전과 학습의 자세를 가져야 한다는 것을 깨달았다. 또한, 동료들과의 소통과 협업이 프로젝트의 성공과 개인의 성장에 큰 영향을 미친다는 것을 알게 되었다. 1년이라는 시간 동안 회사에서의 경험은 나에게 많은 것을 가르쳐주었다. 어려움과 실패의 순간들도 있었지만, 그것들을 통해 더욱 강해지고 성장할 수 있었다. 앞으로 더 많은 도전과 성장을 향해 나아가며, 개발자로서의 역량을 더욱 키워나가고 싶다.'''

In [70]:
diary2

'1년이라는 시간 동안 개발자로서 회사에 다니며 많은 경험과 성장을 한 해였다. 첫날부터 회사의 분위기와 동료들의 친절함에 감동받았고, 새로운 기술과 프로젝트에 도전하는 과정에서 많은 것을 배웠다. 처음에는 많은 어려움과 새로운 지식에 대한 부담을 느꼈지만, 동료들의 도움과 지도를 받으며 조금씩 성장해나갔다. 프로젝트마다 다양한 도구와 언어를 사용하면서 자신의 역량을 끌어올리는 기회가 되었다. 또한, 팀원들과의 협업을 통해 문제를 해결하고 목표를 달성하는 과정에서 많은 것을 배웠다. 한 해 동안 회사에서의 업무와 개발 프로세스에 대해 많은 경험을 쌓았다. 프로젝트 기획부터 개발, 테스트, 배포까지의 과정을 경험하면서 개발자로서의 역할과 책임감을 느낄 수 있었다. 또한, 버그를 해결하거나 기능을 개선하는 과정에서 문제 해결 능력을 키워나갈 수 있었다. 회사 생활을 하면서 가장 중요한 것은 항상 새로운 것을 배우고 개발자로서 성장하는 것이라는 것을 느꼈다. 기술은 빠르게 변하기 때문에 항상 새로운 도전과 학습의 자세를 가져야 한다는 것을 깨달았다. 또한, 동료들과의 소통과 협업이 프로젝트의 성공과 개인의 성장에 큰 영향을 미친다는 것을 알게 되었다. 1년이라는 시간 동안 회사에서의 경험은 나에게 많은 것을 가르쳐주었다. 어려움과 실패의 순간들도 있었지만, 그것들을 통해 더욱 강해지고 성장할 수 있었다. 앞으로 더 많은 도전과 성장을 향해 나아가며, 개발자로서의 역량을 더욱 키워나가고 싶다.'

In [71]:
diary_summary(diary2)

'회사에서의에서의 경험은 나에게 많은 가르침을 주었다. 새로운 기술과 프로젝트에 도전하고 성장하는 과정에서 많은 것을 배웠다. 회사에서의 업무와 개발 프로세스에 대한 경험은 항상 새로운 도전과 학습의 자세를 가져야 한다는 것을 깨달았다.'

In [72]:
make_tag(diary_summary(diary2))

'#회사에서의경험 #새로운도전 #프로젝트도전 #성장과발전 #새로운도전'

In [73]:
txt = "회사에서 중요한 발표를 해야 하는 날이었다. 준비 기간 동안 불안과 긴장으로 가득 찼지만, 동료들의 지원과 격려로 마음을 다잡을 수 있었다. 발표를 하기 전에 몇 차례 연습을 통해 자신감을 끌어올렸고, 결국 발표는 성공적으로 마무리되었다. 발표 이후에는 다른 업무들과의 조화로 바쁜 일정을 소화해야 했다. 프로젝트들의 진행 상황을 체크하고 팀원들과의 회의를 진행하며 업무를 조율하였다. 동료들과의 협업을 통해 문제를 해결하고 프로젝트 목표를 달성하는 과정에서 뿌듯함을 느낄 수 있었다. 하루가 저물어가는 동안, 회사에서의 한 해 동안의 성과와 도전을 돌아보았다. 많은 어려움과 고난이 있었지만, 그것들을 극복하고 성장할 수 있는 기회로 삼을 수 있었다. 회사에서 배운 것들과 동료들과의 소중한 인연은 나에게 큰 보탬이 되었다. 이제 한 해가 끝나가는데, 다가오는 새해에는 더 나은 개발자로 성장하기 위해 노력하고 싶다. 새로운 기술과 도구에 대한 학습을 꾸준히 이어가며, 동료들과의 협업을 통해 더 큰 프로젝트를 성공적으로 이끌어내고 싶다."

In [74]:
txt

'회사에서 중요한 발표를 해야 하는 날이었다. 준비 기간 동안 불안과 긴장으로 가득 찼지만, 동료들의 지원과 격려로 마음을 다잡을 수 있었다. 발표를 하기 전에 몇 차례 연습을 통해 자신감을 끌어올렸고, 결국 발표는 성공적으로 마무리되었다. 발표 이후에는 다른 업무들과의 조화로 바쁜 일정을 소화해야 했다. 프로젝트들의 진행 상황을 체크하고 팀원들과의 회의를 진행하며 업무를 조율하였다. 동료들과의 협업을 통해 문제를 해결하고 프로젝트 목표를 달성하는 과정에서 뿌듯함을 느낄 수 있었다. 하루가 저물어가는 동안, 회사에서의 한 해 동안의 성과와 도전을 돌아보았다. 많은 어려움과 고난이 있었지만, 그것들을 극복하고 성장할 수 있는 기회로 삼을 수 있었다. 회사에서 배운 것들과 동료들과의 소중한 인연은 나에게 큰 보탬이 되었다. 이제 한 해가 끝나가는데, 다가오는 새해에는 더 나은 개발자로 성장하기 위해 노력하고 싶다. 새로운 기술과 도구에 대한 학습을 꾸준히 이어가며, 동료들과의 협업을 통해 더 큰 프로젝트를 성공적으로 이끌어내고 싶다.'

In [75]:
diary_summary(txt)

'회사에서의 한 해 동안의 성과와 도전을 돌아보는 날이었다. 발표를 통해 자신감을 끌어올리고, 다른 업무들과의 조화로 바쁜 일정을 소화해야 했으며, 동료들과의 협업을 통해 더 큰 프로젝트를 성공적으로 이끌어내고 싶다.'

In [76]:
make_tag(diary_summary(txt))

'#회사성과 #도전 #자신감 #동료협력 #성공적프로젝트'

In [77]:
txt = '''가족과 함께 동물원에 간 날이었다! 아침 일찍 일어나서 설레는 마음으로 준비를 마치고, 동물원으로 향했다. 동물원에 도착하자마자 큰 입구에서 우리를 맞이한 사자의 포효 소리가 들려와서 너무 놀랐다. 그리고 사자들의 멋진 모습과 우리에게 호기심 가득한 눈빛이 정말 인상적이었다! 동물원 안에는 다양한 동물들이 있었는데, 코끼리, 원숭이, 퓨마, 얼룩말 등 다양한 동물들을 볼 수 있었다. 특히 코끼리들은 크고 강인한 몸짓으로 우리를 맞이해주어서 더욱 흥미로웠다. 그리고 원숭이들은 가지고 노는 모습이 너무 귀엽고 재미있었다! 동물원에서는 동물들을 관찰하고 알아가는 시간을 가졌다. 동물들의 생태와 특징을 배우고, 어떻게 사는지에 대해 생각해보는 시간이었다. 동물들의 다양한 특성과 생태에 대해 알아가면서 자연에 대한 이해도 높아지고, 동물들을 더욱 더 존중하게 되었다. 동물원을 둘러보며 아이들은 신나게 뛰어다니고, 동물들과의 귀여운 모습을 사진에 담기도 했다. 가족들과 함께 보낸 소중한 시간이었고, 아이들이 많은 것을 배우고 느낄 수 있는 기회가 되었다. 동물원은 언제나 새로운 경험과 발견이 있는 곳이라는 것을 느꼈다. 오늘 동물원에서 보고 느낀 것들은 정말 소중하다. 동물들의 아름다움과 자연의 신비로움을 느끼며, 우리가 지구를 함께 공유하고 있는 동물들과의 조화로운 삶을 생각하게 되었다. 앞으로도 동물들을 잘 챙기고 보호하는 것이 중요하다는 것을 깨달았다.'''

In [78]:
txt

'가족과 함께 동물원에 간 날이었다! 아침 일찍 일어나서 설레는 마음으로 준비를 마치고, 동물원으로 향했다. 동물원에 도착하자마자 큰 입구에서 우리를 맞이한 사자의 포효 소리가 들려와서 너무 놀랐다. 그리고 사자들의 멋진 모습과 우리에게 호기심 가득한 눈빛이 정말 인상적이었다! 동물원 안에는 다양한 동물들이 있었는데, 코끼리, 원숭이, 퓨마, 얼룩말 등 다양한 동물들을 볼 수 있었다. 특히 코끼리들은 크고 강인한 몸짓으로 우리를 맞이해주어서 더욱 흥미로웠다. 그리고 원숭이들은 가지고 노는 모습이 너무 귀엽고 재미있었다! 동물원에서는 동물들을 관찰하고 알아가는 시간을 가졌다. 동물들의 생태와 특징을 배우고, 어떻게 사는지에 대해 생각해보는 시간이었다. 동물들의 다양한 특성과 생태에 대해 알아가면서 자연에 대한 이해도 높아지고, 동물들을 더욱 더 존중하게 되었다. 동물원을 둘러보며 아이들은 신나게 뛰어다니고, 동물들과의 귀여운 모습을 사진에 담기도 했다. 가족들과 함께 보낸 소중한 시간이었고, 아이들이 많은 것을 배우고 느낄 수 있는 기회가 되었다. 동물원은 언제나 새로운 경험과 발견이 있는 곳이라는 것을 느꼈다. 오늘 동물원에서 보고 느낀 것들은 정말 소중하다. 동물들의 아름다움과 자연의 신비로움을 느끼며, 우리가 지구를 함께 공유하고 있는 동물들과의 조화로운 삶을 생각하게 되었다. 앞으로도 동물들을 잘 챙기고 보호하는 것이 중요하다는 것을 깨달았다.'

In [79]:
diary_summary(txt)

'가족과 함께 동물원에서 보고 느낀 것은 정말 소중하고 신나는 동물들의 아름다움과 자연의 신비로움을 느꼈다. 동물들의 다양한 특성과 생태에 대해 알아가는 시간을 가졌다. 동물들을 관찰하고 알아가는 과정에서 자연에 대한 이해와 동물들을 존중하는 중요성을 깨달았다.'

In [80]:
make_tag(diary_summary(txt))

'#가족과함께 #동물원 #아름다움과자연 #자연과함께 #동물존중'